In [1]:
import nltk, string
import numpy as np
import ast
import pickle
import os
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

#### Luigi migration. Task yet to be created

In [27]:
import os
import sys
module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

pars = load_parameters()

so_dump_processed_file = pars.get('input')
output_file = pars.get('output')

code_snippet_col = pars.get('code_snippet_col')
id_col = pars.get('id_col')

api_doc_file = pars.get('api_doc_file')
dataset = pars.get('dataset')
cosine_sim_th = pars.get('cosine_sim_th')

debug = pars.get('debug')

ImportError: No module named 'jupyter_notebook'

#### Utility to get similarity between two texts

In [2]:
def removeSpecialChars(text):
    return re.sub("[^a-zA-Z0-9]", " ", text)

stop_words = set(stopwords.words('english')) 
ps =PorterStemmer()

# Load Doc2Vec model
doc2VecModel = Doc2Vec.load('doc2vec_title.model')

def preprocessText(text):
    text_without_spchar = removeSpecialChars(text)
    word_tokens = word_tokenize(text_without_spchar) 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 

    Stem_words = []
    for w in filtered_sentence:
        rootWord=ps.stem(w)
        Stem_words.append(rootWord)
    return Stem_words

def get_cosine_sim(text1, text2):
    preprocessed_text1 = preprocessText(text1)
    preprocessed_text2 = preprocessText(text2)
    vector1 = doc2VecModel.infer_vector(preprocessed_text1)
    vector2 = doc2VecModel.infer_vector(preprocessed_text2)
    dot = np.dot(vector1, vector2)
    norma = np.linalg.norm(vector1)
    normb = np.linalg.norm(vector2)
    cos = dot / (norma * normb)
    return cos

#### Utility to extract method name

In [3]:
from collections import deque

# Class to extract method as well as attribute calls. Each token after '.' is called attribute 
# be it function call or anything else

class AttributeVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_all_calls(tree):
    all_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Attribute):
            callvisitor = AttributeVisitor()
            callvisitor.visit(node)
            all_calls.append((callvisitor.name, callvisitor.lineno))
    return all_calls

# Visitin method calls only
class FunctionCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FunctionCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append((callvisitor.name, callvisitor.lineno))
    return func_calls

#### Load APIDoc and Dataset

In [5]:
#api_doc_file = '../../data-import/build_api_doc_base/api_doc.csv'
#so_dump_processed_file = '../../../data/stack-overflow/pandas-preprocessedcode-dataset-part3'
#output_file = '../../../data/stack-overflow/pandas-solutioncode-m1'
#code_snippet_col = 'PreprocessedCode3'
#id_col = 'Id'
#cosine_sim_th = 0.0
#dataset = '../../../data/stack-overflow/Dataset - Pandas.csv'

api_df = pd.read_csv(api_doc_file, encoding='ISO-8859-1', error_bad_lines=False)
dataset_df = pd.read_csv(dataset, encoding='ISO-8859-1', error_bad_lines=False)

processed_stackoverflow_df = pd.read_pickle(so_dump_processed_file)


## Get API description with fully qualified name for a method from API doc and build the context
def buildAPIDictionary(api_df):
    api_dict = dict()
    try:
        
        for index, row in api_df.iterrows():
            try:
                r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
                r.extract_keywords_from_text(row['Description'])
                #methodContext[answerId] = ' '.join(r.get_ranked_phrases()[:1]])
                methodContext = row['Description']
            except:
                methodContext = row['Description']
            
            tokens = row['FullyQualifiedName'].split('.')
        
            for token in tokens:
                methodContext = str(methodContext)+' '+token
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print(e)
    return api_dict

def buildAPIDictionaryH2(api_df):
    api_dict_h2 = dict()
    try:
        
        for index, row in api_df.iterrows():
            methodContext = row['SubCategory']
            #tokens = row['FullyQualifiedName'].split('.')
        
            #for token in tokens:
               #methodContext = str(methodContext)+' '+token
            api_dict_h2[row['MethodName']] = methodContext
    except Exception as e:
        print('Error in method buildAPIDictionary',e)
    return api_dict_h2
        
## Get AnswerId and Question Text combo from dataset to build the context
def buildAnswerIdQuestionTextDict(dataset_df):
    dataset_answerId_QText_Dict = dict()
    try:
        for idx, row in dataset_df.iterrows():
            answerId = row['AnswerId']
            if answerId != 0:
                try:
                    r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
                    r.extract_keywords_from_text(row['QuestionText'])
                    dataset_answerId_QText_Dict[answerId] = ' '.join(r.get_ranked_phrases()[:1])
                except:
                    dataset_answerId_QText_Dict[answerId] = row['QuestionText']
    except Exception as e:
        print(e)
        
    return dataset_answerId_QText_Dict

api_dict = buildAPIDictionary(api_df)
api_dict_h2 = buildAPIDictionaryH2(api_df)
dataset_answerId_QText_Dict = buildAnswerIdQuestionTextDict(dataset_df)

In [6]:
def lookUpAPIDocForContext(method_name):
    if method_name in api_dict.keys():
        return api_dict[method_name]
    else:
        return ""

def getSOContext(answerId):
    return dataset_answerId_QText_Dict[int(answerId)]

In [ ]:
def lookUpAPIDocForContextH1H2(method_name):
    try:
        if method_name in api_dict_h2.keys():
            if api_dict_h2[method_name] == "Constructor":
                return False
            else:
                return True
        else:
            return False
    except Exception as e:
        print('Error in method lookUpAPIDocForContext', e)

In [7]:
def M1(df, cosine_sim_threshould):
    
    try:
        # Do not process questions
        if df.PostTypeId == 1:
            df['Solution'] = 'NA'

        else:
            # Parse code and inspect the function
            code_snippet = df[code_snippet_col]
            Id = df[id_col]
            tree = ast.parse(code_snippet)        
            all_calls = get_all_calls(tree)

            solution_lines = set()
            for call, lineno in all_calls:   
               
                tokens = call.split('.')
                method_name = tokens[len(tokens)-1]
                if lookUpAPIDocForContextH1H2(method_name):
                    API_Context = lookUpAPIDocForContext(method_name)
                
                    # If API context is not defined, i.e method is not present in the API, 
                    # reject the method as it could not be solution as per our assumption by making cosine score negative
                    if API_Context == "":
                                cos_score = -1 # Reject the method
                
                    # If method is in API, use its description and match with question text (SO_context)
                    else:
                        SO_Context = getSOContext(Id)
                        cos_score = get_cosine_sim(API_Context, SO_Context)
                    if cos_score > cosine_sim_threshould:
                        #print(API_Context, '\n', SO_Context,  '\n\n')
                        # Use a set to not add the same line twice
                        solution_lines.add(lineno - 1)

            solution = []
            snippet_per_line = code_snippet.split(os.linesep)
            solution_lines = sorted(solution_lines)
            for i in solution_lines:
                solution.append(snippet_per_line[i])


            #print(solution)
            df['Solution'] = solution
            #df['Solution'] = os.linesep.join(solution)
    except KeyError:
        pass
    return df


solution_df = processed_stackoverflow_df.apply(M1, args=(cosine_sim_th,), axis=1)

In [8]:
debug = True

solution_df.to_pickle(output_file)

if debug:
    solution_df.to_csv(output_file + ".csv", sep=",", doublequote=True, index=False)